In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim  

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS

from textblob import TextBlob

from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

C:\Users\HP\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\Users\HP\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


In [2]:
df1 = pd.read_csv('ethiopia_tripadvisor_data_model_3.csv')
df1.head()

,Destination Name,Destination Type,Review Head,Review Body,Review Text,RB_Sentence1,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text,TextBlob Polarity RBS1,RB_Sentence1_Split
0,Rock-Hewn Churches of Lalibela,Landmarks,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,Amazing location must visit.,-0.05,0.300000,0.125000,0.6000,"['Amazing', 'location', 'must', 'visit.']"
1,Rock-Hewn Churches of Lalibela,Landmarks,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,I do recommend to visit Lalibela churches and ...,0.25,-0.150000,-0.016667,0.0000,"['I', 'do', 'recommend', 'to', 'visit', 'Lalib..."
2,Rock-Hewn Churches of Lalibela,Landmarks,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,A must visit in Ethiopia - one of the most uni...,0.90,0.345000,0.483750,0.4375,"['A', 'must', 'visit', 'in', 'Ethiopia', '-', ..."
3,Rock-Hewn Churches of Lalibela,Landmarks,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,These churches have to be seen to be believed.,0.60,0.225000,0.350000,0.0000,"['These', 'churches', 'have', 'to', 'be', 'see..."
4,Rock-Hewn Churches of Lalibela,Landmarks,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,Thanks to our tour organiser Ephram who is bas...,0.20,0.233333,0.228571,0.2000,"['Thanks', 'to', 'our', 'tour', 'organiser', '..."


In [3]:
df1['RH_Split'] = df1['Review Head'].apply(lambda x: x.split(' '))
df1['RB_Split'] = df1['Review Body'].apply(lambda x: x.split(' '))
df1['RT_Split'] = df1['Review Text'].apply(lambda x: x.split(' '))
df1['RB_Sentence1_Split'] = df1['RB_Sentence1'].apply(lambda x: x.split(' '))
df1.head()

,Destination Name,Destination Type,Review Head,Review Body,Review Text,RB_Sentence1,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text,TextBlob Polarity RBS1,RB_Sentence1_Split,RH_Split,RB_Split,RT_Split
0,Rock-Hewn Churches of Lalibela,Landmarks,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,Amazing location must visit.,-0.05,0.300000,0.125000,0.6000,"[Amazing, location, must, visit.]","[Incredible, locations, don't, be, put, off, t...","[Amazing, location, must, visit., This, site, ...","[Incredible, locations, don't, be, put, off, t..."
1,Rock-Hewn Churches of Lalibela,Landmarks,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,I do recommend to visit Lalibela churches and ...,0.25,-0.150000,-0.016667,0.0000,"[I, do, recommend, to, visit, Lalibela, church...","[It, is, a, real, miracle, of, the, world!]","[I, do, recommend, to, visit, Lalibela, church...","[It, is, a, real, miracle, of, the, world!, I,..."
2,Rock-Hewn Churches of Lalibela,Landmarks,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,A must visit in Ethiopia - one of the most uni...,0.90,0.345000,0.483750,0.4375,"[A, must, visit, in, Ethiopia, -, one, of, the...","[Incredible, experience, ]","[A, must, visit, in, Ethiopia, -, one, of, the...","[Incredible, experience, A, must, visit, in, E..."
3,Rock-Hewn Churches of Lalibela,Landmarks,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,These churches have to be seen to be believed.,0.60,0.225000,0.350000,0.0000,"[These, churches, have, to, be, seen, to, be, ...",[Amazing],"[These, churches, have, to, be, seen, to, be, ...","[Amazing, These, churches, have, to, be, seen,..."
4,Rock-Hewn Churches of Lalibela,Landmarks,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,Thanks to our tour organiser Ephram who is bas...,0.20,0.233333,0.228571,0.2000,"[Thanks, to, our, tour, organiser, Ephram, who...","[So, much, history]","[Thanks, to, our, tour, organiser, Ephram, who...","[So, much, history, Thanks, to, our, tour, org..."


In [4]:
df1['Destination Type'].value_counts()

Landmarks       3803
Museums         2507
Nature/Parks    2001
Name: Destination Type, dtype: int64

### Topic Modelling

In [5]:
# NLTK Stop words
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

test_subset = df1['RB_Sentence1_Split'].values
sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for value in test_subset:
    for word in value:
        if (sid.polarity_scores(word)['compound']) >= 0.4:
            pos_word_list.append(word)
        elif (sid.polarity_scores(word)['compound']) <= -0.4:
            print
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)                

print('Positive :', len(pos_word_list))       
print('Neutral :', len(neu_word_list))    
print('Negative :', len(neg_word_list)) 

Positive : 5775
Neutral : 133685
Negative : 748


In [6]:
from nltk.corpus import stopwords

main_stop_words = ['ethiopia', 'ethiopian', 'rock', 'hewn', 'church', 'churches', 'park', 'parks', 'museum', 
                       'museums', 'omo', 'valley', 'mountain', 'mountains', 'addis', 'ababa', 'lucy', 'simien', 
                       'national', 'lalibela', 'gelada', 'haile', 'selassie', 'gondar', 'africa' , 'harar', 
                       'cathedral', 'cathedrals', 'place', 'visit', 'girmay', 
                       '\n', '‘s']
positive_stop_words = list(set(pos_word_list))
negative_stop_words = list(set(neg_word_list))
total_stop_words = main_stop_words + positive_stop_words + negative_stop_words

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.extend(total_stop_words)
print('Stop Words: ', len(stop_words))

Stop Words:  1148


In [7]:
# Convert to list
data = df1[df1['Destination Type'] == 'Landmarks'][df1['TextBlob Polarity RBS1'] > 0]['RB_Sentence1'].values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Amazing location must visit.']


In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['amazing', 'location', 'must', 'visit']]


In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['amazing', 'location', 'must', 'visit']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['location']]


In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1)]]


In [13]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('location', 1)]]

In [14]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [15]:
# Print the Keyword in the n topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.072*"wall" + 0.068*"village" + 0.048*"inside" + 0.034*"still" + '
  '0.025*"especially" + 0.025*"holy" + 0.023*"piece" + 0.022*"use" + '
  '0.019*"outside" + 0.019*"ceiling"'),
 (1,
  '0.167*"see" + 0.083*"history" + 0.054*"city" + 0.045*"monastery" + '
  '0.030*"find" + 0.030*"important" + 0.022*"orthodox" + 0.021*"drive" + '
  '0.014*"art" + 0.014*"travel"'),
 (2,
  '0.138*"visit" + 0.056*"day" + 0.047*"look" + 0.038*"stone" + 0.025*"top" + '
  '0.025*"carve" + 0.022*"famous" + 0.022*"spend" + 0.022*"live" + '
  '0.018*"visitor"'),
 (3,
  '0.112*"guide" + 0.063*"local" + 0.057*"much" + 0.049*"also" + 0.032*"lot" + '
  '0.029*"place" + 0.023*"modern" + 0.022*"aksum" + 0.019*"thing" + '
  '0.018*"priest"'),
 (4,
  '0.086*"people" + 0.052*"walk" + 0.047*"well" + 0.044*"ancient" + '
  '0.041*"building" + 0.038*"know" + 0.030*"show" + 0.028*"climb" + '
  '0.025*"come" + 0.022*"can"'),
 (5,
  '0.122*"worth" + 0.051*"way" + 0.029*"cave" + 0.028*"ground" + 0.022*"boat" '
  '+ 0.020

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.954865539425064

Coherence Score:  0.5045791204467835


In [17]:
# Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

In [18]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
import os
os.environ.update({'MALLET_HOME':r'C:/Users/HP/mallet-2.0.8/'})
mallet_path = r'C:\Users\HP\mallet-2.0.8\bin\mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [19]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('ancient', 0.05101058710298364),
   ('find', 0.04523580365736285),
   ('building', 0.03946102021174206),
   ('give', 0.03464870067372473),
   ('local', 0.03176130895091434),
   ('ground', 0.028873917228103944),
   ('part', 0.026948989412897015),
   ('huge', 0.02598652550529355),
   ('large', 0.023099133782483156),
   ('early', 0.023099133782483156)]),
 (1,
  [('history', 0.14300202839756593),
   ('site', 0.07099391480730223),
   ('lot', 0.04766734279918864),
   ('architecture', 0.037525354969574036),
   ('main', 0.036511156186612576),
   ('structure', 0.032454361054766734),
   ('century', 0.018255578093306288),
   ('reason', 0.018255578093306288),
   ('story', 0.018255578093306288),
   ('ceremony', 0.017241379310344827)]),
 (2,
  [('walk', 0.0599250936329588),
   ('small', 0.05337078651685393),
   ('year', 0.04681647940074907),
   ('tourist', 0.027153558052434457),
   ('historic', 0.023408239700374533),
   ('back', 0.02247191011235955),
   ('expect', 0.020599250936329586),
   

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

start1 = 2 
limit1 = 11
step1 = 1

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start1, limit=limit1, step=step1)

x = range(start1, limit1, step1)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [145]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.3692
Num Topics = 3  has Coherence Value of 0.3985
Num Topics = 4  has Coherence Value of 0.4567
Num Topics = 5  has Coherence Value of 0.5056
Num Topics = 6  has Coherence Value of 0.4873
Num Topics = 7  has Coherence Value of 0.5551
Num Topics = 8  has Coherence Value of 0.5342
Num Topics = 9  has Coherence Value of 0.5415
Num Topics = 10  has Coherence Value of 0.5542


In [20]:
# optimal_model = model_list[5] # Number of Topics = 7

optimal_model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=7, id2word=id2word)
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.071*"experience" + 0.063*"people" + 0.060*"build" + 0.038*"small" + '
  '0.034*"year" + 0.032*"incredible" + 0.030*"place" + 0.026*"cave" + '
  '0.022*"full" + 0.022*"unique"'),
 (1,
  '0.094*"history" + 0.053*"trip" + 0.033*"religious" + 0.029*"town" + '
  '0.025*"famous" + 0.024*"main" + 0.023*"travel" + 0.021*"important" + '
  '0.020*"live" + 0.017*"part"'),
 (2,
  '0.083*"time" + 0.054*"make" + 0.043*"painting" + 0.031*"lot" + '
  '0.025*"thing" + 0.024*"visit" + 0.020*"absolutely" + 0.020*"work" + '
  '0.017*"ceiling" + 0.016*"enter"'),
 (3,
  '0.119*"visit" + 0.082*"day" + 0.033*"find" + 0.030*"historical" + '
  '0.028*"building" + 0.021*"spectacular" + 0.019*"spend" + 0.017*"large" + '
  '0.017*"locate" + 0.017*"feel"'),
 (4,
  '0.082*"guide" + 0.061*"tour" + 0.033*"inside" + 0.031*"city" + 0.025*"give" '
  '+ 0.024*"area" + 0.021*"tourist" + 0.021*"ground" + 0.019*"priest" + '
  '0.019*"picture"'),
 (5,
  '0.109*"worth" + 0.043*"walk" + 0.034*"local" + 0.020*"hour" + 

In [21]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,6.0,0.1597,"site, carve, ancient, top, world, view, stone,...",Amazing location must visit.
1,1,0.0,0.1694,"experience, people, build, small, year, incred...",A must visit in Ethiopia - one of the most uni...
2,2,4.0,0.1575,"guide, tour, inside, city, give, area, tourist...",Thanks to our tour organiser Ephram who is bas...
3,3,0.0,0.1553,"experience, people, build, small, year, incred...",Truly a marvel of early architecture and craft...
4,4,2.0,0.1827,"time, make, painting, lot, thing, visit, absol...","Please I insist anyone reaches Ethiopia,, neve..."
5,5,0.0,0.1429,"experience, people, build, small, year, incred...",The rock hewn churches are just amazing.
6,6,1.0,0.1597,"history, trip, religious, town, famous, main, ...",Easy access to churches from Lilibela Town Cen...
7,7,2.0,0.1955,"time, make, painting, lot, thing, visit, absol...",Lalibela has turned out as one of my favourite...
8,8,0.0,0.1566,"experience, people, build, small, year, incred...",Just got back from an amazing trip to Ethiopia.
9,9,3.0,0.1704,"visit, day, find, historical, building, specta...",Did enjoy scrabbling around these churches - f...


In [22]:
df_dominant_topic['Dominant_Topic'].value_counts()

0.0    693
1.0    371
2.0    359
5.0    296
3.0    294
4.0    293
6.0    270
Name: Dominant_Topic, dtype: int64

In [29]:
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 0]['Text'].values # General Destination Appreciation - 693
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 1]['Text'].values # General Destination Appreciation - 371
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 2]['Text'].values # General Destination Appreciation - 359
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 3]['Text'].values # General Destination Appreciation - 296
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 4]['Text'].values # Tour Guide - 294
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 5]['Text'].values # Visual (Aesthetic, Architecture, etc.) - 293
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 6]['Text'].values # General Destination Appreciation - 270

array(['Amazing location must visit.',
       'The site of these rock hewn churches is one of the most spectacular and rare evidence of Africas historic civilizations.',
       'As a UNESCO World Heritage site, you already know its going to be something special.',
       'These rock churches have been on the top of my bucket list and they were nothing short of astounding.',
       'I thoroughly enjoyed visiting 11 amazing churches carved out of stone by King Lalibela.',
       'Visit with the other churches be a good walker, ther is a lot of climbing and talking off of shoes.',
       'If you are planning to make your holiday to Ethiopia, Rockhewn Churches of Lalibela are one of the most important attraction site that you have to see.',
       'Lalibela was on my bucket list for ages but I have to admit having been in Jordan seeing Petra this year as well - Lalibela was not that impressive .',
       'Amazing place, definitely a world wonder.',
       'A beautiful cross-shaped rock-hew

In [30]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.2265,"experience, people, build, small, year, incred...","This church, built in an enormous cave in the ..."
1,1.0,0.2391,"history, trip, religious, town, famous, main, ...",This pool is the main one used for the Timket ...
2,2.0,0.2263,"time, make, painting, lot, thing, visit, absol...",The whole ceiling is covered with the amazing ...
3,3.0,0.2442,"visit, day, find, historical, building, specta...",Spent a day visiting Lalibela churches which I...
4,4.0,0.2400,"guide, tour, inside, city, give, area, tourist...","If you can locate a deacon to give you a tour,..."
5,5.0,0.2355,"worth, walk, local, hour, wall, drive, country...",The drive there from Lalibela town to the moni...
6,6.0,0.2300,"site, carve, ancient, top, world, view, stone,...",the monastery found very close to baher dar to...


In [31]:
sent_topics_sorteddf_mallet['Text'].values

array(['This church, built in an enormous cave in the 11th Century is a masterpiece - and formed the pattern for the unique Ethiopian style of rock-hewn churches (hewn down into the ground) that were created over the next few hundred years.',
       'This pool is the main one used for the Timket (Epiphany) ceremony and thousands of people gather around it to participate at the religious ceremonies !',
       'The whole ceiling is covered with the amazing painted angels that are now ubiquitous throughout Ethiopia - from coffee cups to traditional clothes to pens to hotel brochures.',
       'Spent a day visiting Lalibela churches which I felt was enough - versus the suggestions from most who suggest 2-3 days.',
       'If you can locate a deacon to give you a tour, you can pay 50 birr ($2.50USD) for a very informative tour of the whole grounds.',
       'The drive there from Lalibela town to the monitory by it self is fantastic, the scenery is amazing with beautiful landscape and farmer

In [32]:
# # Number of Documents for Each Topic
# topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# # Percentage of Documents for Each Topic
# topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# # Topic Number and Keywords
# topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# # Concatenate Column wise
# df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# # Change Column names
# df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# # Show
# df_dominant_topics